In [1]:
# Impoorted For Scraping Tweets
import tweepy

# Pretty Printing JSON data
import json

# Making the Dataframes
import pandas as pd

# Importing the API keys
from api_keys import consumer_key , consumer_secret , access_token , access_token_secret

# Importing the model functions
from model import predict_sentiment , process_tweet

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Set-up for Tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [3]:
#User
user_names = []

file_in = open('users.txt','r')
for line in file_in:
    user_names.append(line.strip('\n'))

## Twit Score Calculation

|Serial No. | Score | Data | Weights |
| -- | -- | -- | -- |
|1|**Friend Follow Ratio**|followers_count / friends_count| 25% of Twit Score|
|2|**Relevance Score**||25% of Twit Score|
|2 (a)|Listed Ratio|listed_count / followers_count|40% of Relevance Score|
|2 (b)|ReTweet|Average retweet count for past tweets|30% of Relevance Score|
|2 (c)|Likes|Average likes for past tweets|30% of Relevance Score|
|3|**Usage Score**||25% of Twit Score|
|3 (a)|Tweet Frequency|Average time between tweets|50% of Usage Score|
|3 (b)|Media|Amount of status posted |30% of Usage Score|
|3 (c)|Twitter Bio| description|10% of Usage Score|
|3 (d)|Profile Picture|profile_image_url|10% of Usage Score|
|4|**Authenticity Score**||25% of Twit Score|
|4 (a)|Duration|created_at|60% of Authenticity Score|
|4 (b)|Followers Count|followers_count|40% of Authenticity Score|
|  |  |  | **100% of Twit Score** |

In [4]:
from datetime import datetime, date
from dateutil import relativedelta

def duration_months(created):
    start = created
    end = datetime.strptime(str(date.today()), "%Y-%m-%d")

    diff = relativedelta.relativedelta(end, start)
    return diff.months + diff.years * 12

In [5]:
data = []

for u in user_names:
    print(u + " fetched!")
    number_of_tweets = 200

    # Fetching the timeline of the user
    timeline = api.user_timeline(id=u, count=number_of_tweets, include_rts=True, tweet_mode = 'extended')

    # Extracting the data of the user
    
    Name = str(timeline[0]._json["user"]["name"])
    Username = str(timeline[0]._json["user"]["screen_name"])
    Follower_Count = int(timeline[0]._json["user"]["followers_count"])
    Following_Count = int(timeline[0]._json["user"]["friends_count"])
    Listed_Count = int(timeline[0]._json["user"]["listed_count"])
    Profile_Picture = str(timeline[0]._json["user"]["profile_image_url"] != "")
    Description = str(timeline[0]._json["user"]["description"] != "")
    Media = int(timeline[0]._json["user"]["statuses_count"])
    Created = api.get_user(u).created_at

    tweets = []
    time = []
    retweets = []
    likes = []

    for tweet in timeline:
        tweets.append(tweet.full_text)
        time.append(tweet.created_at)
        retweets.append(tweet.retweet_count)
        likes.append(tweet.favorite_count)

    data.append({
        "Name" : Name,
        "Username" : Username,
        "FollowerCount" : Follower_Count,
        "FollowingCount" : Following_Count,
        "ListedCount" : Listed_Count,
        "Active" : duration_months(Created),
        "TotalTweets" : Media,
        "ProfilePicture" : Profile_Picture,
        "Description" : Description,
        "TweetList" : tweets,
        "TimeList" : time,
        "Retweets" : retweets,
        "Likes" : likes
    })

junaidrahxm fetched!
bhatvakash fetched!
ShyamSu06954329 fetched!
godfather01R fetched!
frugaltrader fetched!
jessi_moorhouse fetched!
DaveRamsey fetched!
preetbanerjee fetched!
ellenroseman fetched!
BoomerandEcho fetched!
behaviorgap fetched!
HowToSave fetched!
Xanedro fetched!
rat_tweak fetched!
pranavmanie fetched!
slowphoton fetched!
iancassel fetched!
jvembuna fetched!
trengriffin fetched!
dmuthuk fetched!
Vivek_Investor fetched!
rohitchauhan fetched!
benshapiro fetched!
KanchanGupta fetched!
mkatju fetched!
Mundra08 fetched!
kprabhdeep fetched!
ramgopal_rao fetched!
agrawalmanindra fetched!
garyvee fetched!
GuyKawasaki fetched!
Julian fetched!
sophiaamoruso fetched!
pourmecoffee fetched!


In [6]:
#Converting to dataframe

dataset = pd.DataFrame(data)

In [7]:
#Tweet Frequency
def tweet_frequency(time_list):
    time_diff_tweets = []

    for i in range(0,len(time_list)-1):
        diff = relativedelta.relativedelta(time_list[i], time_list[i+1])
        diff_min = diff.minutes + diff.hours*60 + diff.days*24*60 + diff.months*30*24*60 + diff.years*365*24*60
        time_diff_tweets.append(diff_min)

    return sum(time_diff_tweets)/(number_of_tweets-1)

In [8]:
def calc_retweet_count(row):
    return sum(row['Retweets'])/(len(row['Retweets']))

def calc_tweet_freq(row):
    return tweet_frequency(row['TimeList'])

def calc_FFratio(row):
    return row['FollowerCount']/row['FollowingCount']

def calc_Listed_ratio(row):
    return row['ListedCount']/row['FollowerCount']

def calc_like_count(row):
    return sum(row['Likes'])/(len(row['Likes']))

In [9]:
dataset['RetweetCount'] = dataset.apply(calc_retweet_count, axis=1)
dataset['TweetFreq'] = dataset.apply(calc_tweet_freq, axis=1)
dataset['FFRatio'] = dataset.apply(calc_FFratio, axis=1)
dataset['ListedRatio'] = dataset.apply(calc_Listed_ratio, axis=1)
dataset['LikeCount'] = dataset.apply(calc_like_count, axis=1)

In [10]:
#Scaling Values
from sklearn.preprocessing import MinMaxScaler

X = dataset.iloc[:, :].values
scaler_1 = MinMaxScaler()
scaler_2 = MinMaxScaler()
X[:, 2:7] = scaler_1.fit_transform(X[:, 2:7])
X[:, 13:] = scaler_2.fit_transform(X[:, 13:])

In [11]:
# Setting up the dataframe

dataset = pd.DataFrame(X, columns = ["Name", "Username","FollowerCount","FollowingCount","ListedCount","Active","TotalTweets","ProfilePicture","Description","TweetList","TimeList","Retweets", "Likes","RetweetCount", "TweetFreq", "FFRatio", "ListedRatio" , "LikeCount"])

In [12]:
#Calculation of Twit Score
def calc_twit(row):
    pic = 0
    bio = 0
    if(row['ProfilePicture'] == "True"):
        pic = 1
        
        if(row['Description'] == "True"):
            bio = 1

            relavance_score = (0.4 * row['ListedRatio']) + (0.3 * row['RetweetCount']) + (0.3 * row['LikeCount'])
            usage_score = (0.5 * row['TweetFreq']) + (0.3 * row['TotalTweets']) + (0.1*(bio + pic))
            authenticity_score = (0.6 * row['Active']) + (0.4 * row['FollowerCount'])

            return ((0.25 * row['FFRatio']) + (0.25 * relavance_score) + (0.25 * usage_score) + (0.25 * authenticity_score))*100

In [14]:
dataset['TwitScore'] = dataset.apply(calc_twit, axis=1)

In [15]:
# Importing the words to check weather a tweet is financial or not

financial_words = set()

file = open('advanced.txt')
for line in file:    
    financial_words.add(line)

file.close()

file = open('common.txt')
for line in file:    
    financial_words.add(line.strip('\n'))

file.close()

In [16]:
financial_total = 0
financial_positive = 0
profile_total = 0
profile_positive = 0
is_financial = False
actual_financial_score = 0

scores = {
    'ProfileScore' : [],
    'FinancialScore' : []
}

for row in range(len(dataset)):
    tweets_list = dataset.iloc[row ,9]
    
    for tweet in tweets_list:
        tweet = process_tweet(tweet)
        
        sentiment = predict_sentiment(tweet)
        sentiment = sentiment[0]
        
        for word in tweet:
            if word in financial_words:
                is_financial = True
                break
                
        if is_financial == True:
            financial_total += 1
            financial_positive += sentiment
            
        is_financial = False
        
        profile_total += 1
        profile_positive += sentiment
        
    scores['ProfileScore'].append(profile_positive / profile_total * 100)
    
    if financial_total == 0:
        scores['FinancialScore'].append(0)
    else:    
        actual_financial_score = 0.5 * ((financial_positive / financial_total * 100) + (financial_total / profile_total * 100))
        scores['FinancialScore'].append(actual_financial_score)
        
    financial_total = 0
    financial_positive = 0
    profile_total = 0
    profile_positive = 0
        

In [17]:
scores = pd.DataFrame(scores)

In [18]:
df = pd.concat([dataset , scores] , axis = 1)

In [19]:
def calc_score(row):
    return row["TwitScore"] + row['FinancialScore'] + row['ProfileScore']

In [20]:
df['BehavioralScore'] = df.apply(calc_score, axis=1)

In [21]:
df

,Name,Username,FollowerCount,FollowingCount,ListedCount,Active,TotalTweets,ProfilePicture,Description,TweetList,...,Likes,RetweetCount,TweetFreq,FFRatio,ListedRatio,LikeCount,TwitScore,ProfileScore,FinancialScore,BehavioralScore
0,Junaid Rahim,junaidrahxm,0.000064,0.061139,0.000163,0.355828,0.006609,True,True,"[@iiverveii @ezyang tru, hehe https://t.co/9ar...",...,"[1, 2, 0, 5, 0, 6, 2, 12, 1, 0]",0.014398,0.54224,0.000006,0.425689,0.000849,21.638631,80.0,55.000000,156.638631
1,Akash Bhat,bhatvakash,0.00074,0.084941,0.004342,0.858896,0.038227,True,True,[@harshitagirase @snigdhasur @PranjalDaga1 @Th...,...,"[2, 2, 1, 0, 0, 0, 2, 0, 1, 3]",0.00005,0.006409,0.000047,1.0,0.000322,28.259574,100.0,75.000000,203.259574
2,Shyam Subba,ShyamSu06954329,0.0,0.014647,0.0,0.0,0.0,True,True,[It's funny how people will spend hours lookin...,...,"[0, 0, 2, 1, 0, 0, 1, 0, 2, 1]",0.0,0.112194,0.0,0.0,0.000205,6.402430,90.0,65.000000,161.402430
3,Rishabh,godfather01R,0.000048,0.024325,0.0,0.104294,0.003594,True,True,[@safiranand Promoter selling stake continuous...,...,"[1, 4, 2, 5, 2, 0, 0, 2, 1, 2]",0.0,0.055454,0.000011,0.0,0.000556,7.285288,80.0,57.500000,144.785288
4,MillionDollarJourney,FrugalTrader,0.002914,0.051396,0.009525,0.969325,0.029601,True,True,[RT @DividendHawk: World's Largest Dividend Pa...,...,"[0, 5, 0, 0, 0, 0, 0, 0, 0, 0]",0.001046,0.354029,0.000309,0.557278,0.000146,29.812570,70.0,63.333333,163.145904
5,"Jessica Moorhouse, AFCC®",jessi_moorhouse,0.00394,0.076179,0.011615,0.693252,0.100754,True,True,[My career path has been anything but linear &...,...,"[1, 0, 0, 0, 5, 2, 2, 2, 1, 1]",0.000598,0.179031,0.000282,0.502477,0.00041,23.472500,90.0,77.857143,191.329643
6,Dave Ramsey,DaveRamsey,0.258565,0.003269,0.17962,0.97546,0.023741,True,True,"[You're the hero. You're not the victim., Get ...",...,"[507, 11, 123, 21, 0, 0, 43, 932, 45, 1006]",0.012854,0.103261,0.422966,0.118449,0.078725,35.637810,90.0,65.000000,190.637810
7,Preet Banerjee,preetbanerjee,0.004624,0.055712,0.012836,0.877301,0.082654,True,True,[RT @erinbury: Thanks for sharing Shomik! Glad...,...,"[0, 1, 1, 2, 1, 1, 0, 9, 1, 3]",0.000249,0.039295,0.000452,0.473191,0.000556,24.063795,90.0,70.000000,184.063795
8,Ellen Roseman,ellenroseman,0.003056,0.174394,0.010475,0.883436,0.068158,True,True,[RT @Iam_Canadian: @67Dodge @habsrgr8 @ellenro...,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0.000399,0.039023,0.000095,0.584207,0.000029,25.131506,70.0,60.000000,155.131506
9,Boomer and Echo,BoomerandEcho,0.003612,0.070817,0.011859,0.760736,0.114132,True,True,[New | Designing Your Retirement Lifestyle: It...,...,"[2, 1, 1, 0, 2, 0, 1, 7, 2, 11]",0.002441,0.070177,0.000278,0.559662,0.000791,23.820551,60.0,63.571429,147.391980


In [23]:
import joblib

joblib.dump(scaler_1, "./static/Scaler_1.mdl")
joblib.dump(scaler_2, "./static/Scaler_2.mdl")

['./static/Scaler_2.mdl']